# **My Week 03 Notes**
This notebook contains my notes on everything I covered in **Week 03** of DS105A.

For this notebook, I will follow the structure of the **W03 Formative** and **W03 Lab** to collect temperature data from the **OpenMetoAPI**. 
## Contents:
- Collecting Data Using API's
    - Automated Data Collection
    - Storing Data to a Dictionary
    - Saving to a .json File
- Data Visualisation
    - Abu Dhabi
    - Madrid


## Imports

In [ ]:
import os
import json

import requests

import pandas as pd

## Collecting Data Using API's
URL: `https://api.open-meteo.com/v1/forecast`

I will use this URL to retrieve historical forecast data for 2 cities: Abu Dhabi and Madrid.

My aim is to create a visualisation for the daily minimum and maximum temperatures from 01/01/2023-31/12/2023 for both locations.

I will also be using the world_cities.csv file to automatically collect the latitudes and longitudes of both cities.

### Automated Data Collection
It is very tedious to rewrite the same code over and over again.

It is possible to define functions in Python that automate repetitive tasks (credit to DS105 team).

The function below read the world cities data from the world_cities.csv file and returns the latitude and longitude of the desired city:

In [ ]:
def get_lat_lon(country_code, city):
    
    filepath = './world_cities.csv'
    world_cities = pd.read_csv(filepath)

    city_data = world_cities[(world_cities['country'] == country_code) & 
                             (world_cities['name'] == city)]
    
    city_data = city_data.to_dict('records')
    
    if len(city_data) == 0:
        raise ValueError(f"No records found for {city}, {country_code} in {filepath}")

    latitude = city_data[0]['lat']
    longitude = city_data[0]['lng']

    return latitude, longitude

This function returns the temperature data for a given location just by passing it the latitude and longitude:

(**Note**: This is the get_forecast_data function found in the W03 formative)

In [ ]:
def get_temperature_data(latitude, longitude, start_date, end_date):

    base_forecast_url = "https://archive-api.open-meteo.com/v1/archive?"
    params_lat_long = "latitude=" + str(latitude) + "&longitude="  + str(longitude)
    params_others = "&start_date=" + start_date + "&end_date=" + end_date + "&daily=temperature_2m_max,temperature_2m_min" + "&timezone=auto"

    final_url = base_forecast_url + params_lat_long + params_others

    response = requests.get(final_url)

    response

    response.status_code
    response.content

    forecast_data = response.json()
    forecast_temperatures = forecast_data['daily']
    return forecast_temperatures

Retrieving **Abu Dhabi's** temperature data:

1) I found the latitude and longitude of Abu Dhabi using the get_lat_lon function

2) Then, I used the get_temperature_data function to fetch Abu Dhabi's temperatures

In [ ]:
latitude, longitude = get_lat_lon("AE", "Abu Dhabi")
print(latitude, longitude)

In [ ]:
get_temperature_data(latitude=24.45118, longitude=54.39696, start_date="2023-01-01", end_date="2023-12-31")

Retrieving **Madrid's** temperature data:

The same steps apply for Madrid!

In [ ]:
latitude, longitude = get_lat_lon("ES", "Madrid")
print(latitude, longitude)

In [ ]:
get_temperature_data(latitude=40.4165, longitude=-3.70256, start_date="2023-01-01", end_date="2023-12-31")

### Storing Data to a Dictionary

Creating a dictionary for **Abu Dhabi**

I created a variable for Abu Dhabi's temperatures between 01/01/2023 and 31/12/2023:

In [ ]:
abudhabi_temperatures = get_temperature_data(latitude=24.45118, longitude=54.39696, start_date="2023-01-01", end_date="2023-12-31")

I then defined Abu Dhabi's location details - The country code as 'AE' and the city as 'Abu Dhabi'.

I also defined the dates, as well as the minimum and maximum temperatures by using the abudhabi_temperatures variable:

In [ ]:
country = "AE"
city = "Abu Dhabi"
date = abudhabi_temperatures["time"]
min_temp = abudhabi_temperatures["temperature_2m_min"]
max_temp = abudhabi_temperatures["temperature_2m_max"]

Lastly, I created a dictionary to organise the data.

The country and city keys store the location information.

Date, Minimum Temperature, and Maximum Temperature store lists of dates and temperature values, making it easier to display the data:

In [ ]:
abudhabi_data = {"Country": country, "City": city, "Date": date, "Minimum Temperature": min_temp, "Maximum Temperature": max_temp}

Creating a dictionary for **Madrid**:

Aaand repeat

In [ ]:
madrid_temperatures = get_temperature_data(latitude=40.4165, longitude=-3.70256, start_date="2023-01-01", end_date="2023-12-31")
country = "ES"
city = "Madrid"
date = madrid_temperatures["time"]
min_temp = madrid_temperatures["temperature_2m_min"]
max_temp = madrid_temperatures["temperature_2m_max"]

In [ ]:
madrid_data = {"Country": country, "City": city, "Date": date, "Minimum Temperature": min_temp, "Maximum Temperature": max_temp}

## Saving Data to a .json File

Saving the **Abu Dhabi** dictionary:

In [ ]:
with open("./abudhabi_temperatures.json", "w") as file:
    json.dump(abudhabi_data, file)

Saving the **Madrid** dictionary:

In [ ]:
with open("./madrid_temperatures.json", "w") as file:
    json.dump(madrid_data, file)

## Data Visualisation
In this section, I will create tables and graphs for each city's data using pandas DataFrame.

### Abu Dhabi
Loading Abu Dhabi's data and creating a DataFrame from the dictionary:

In [ ]:
with open("./abudhabi_temperatures.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)

The code below returns the first 5 rows of the DataFrame:

In [ ]:
df.head()

Conversely, this code returns the last 5 rows:

In [ ]:
df.tail()

Finally, the .plot() function can be used to plot graphs using data from the table.

In this case, the .plot() function will plot Abu Dhabi's minimum and maximum temperatures:

In [ ]:
df.plot(x='Date', y=['Minimum Temperature', 'Maximum Temperature'], figsize=(20, 10))

### Madrid

In [ ]:
with open("./madrid_temperatures.json") as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.plot(x="Date", y=["Minimum Temperature", "Maximum Temperature"], figsize=(20, 10))